In [4]:
import cv2 as cv
from pynput.mouse import Button, Controller
from pynput import mouse
import numpy as np
import keyboard
import time

In [3]:
numPresses=0
#starting x and y
sX,sY=0,0
#ending x and y
eX,eY=0,0
#controller (mouse)
cont = Controller()

#on click listener from documentation
def on_click(x, y, button, pressed):
    global sX,sY
    global eX,eY
    global numPresses
    if pressed:
        if numPresses==0:
            sX,sY=cont.position
            numPresses+=1
        if numPresses==1:
            eX,eY=cont.position
    if not pressed:
        # Stop listener
        return False
    
#get the starting point and ending point
with mouse.Listener(on_click=on_click) as listener:
    listener.join()
with mouse.Listener(on_click=on_click) as listener:
    listener.join()

#variable for scale computation
var=sY-eY

In [7]:
#image location
img = cv.imread('Insert Image Location')

#calculate image scaling through starting and ending point
scale = (abs(var)/img.shape[0]) 

#orig width in img.shape[1]
#orig height in img.shape[0]
width = int(img.shape[1] * scale)
height = int(img.shape[0] * scale)
dimensions = (width, height)
  
# resize image
resized = cv.resize(img, dimensions, interpolation = cv.INTER_AREA)

#grayscale image
imgray = cv.cvtColor(resized,cv.COLOR_BGR2GRAY)

#blurred image
imgblur=cv.GaussianBlur(imgray, (5, 5), 1.4)

#canny edge detection
edges = cv.Canny(imgray, threshold1=30, threshold2=100)

#coordinates of edges
indices = np.where(edges != [0])
coordinates = list(zip(indices[0], indices[1]))

#press to start drawing
with mouse.Listener(on_click=on_click) as listener:
    listener.join()
#returns mouse to starting position
currx,curry=cont.position
cont.move(sX-currx,sY-curry)

for i in range(len(coordinates)):
    #press esc to exit program
    if keyboard.is_pressed('Esc'):
        break
    else:
        #returns mouse to start position
        currx,curry=cont.position
        cont.move(sX-currx,sY-curry)
        
        #move mouse to each point and draws each point
        cont.move(coordinates[i][1],coordinates[i][0])
        cont.click(Button.left)
        time.sleep(0.000000000000000000000000000000005)
        
# cv.imshow('Image',resized)
# cv.imshow('Image Gray',edges)
# cv.waitKey(0)
# cv.destroyAllWindows()